In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df1=pd.read_pickle("tw_pos.pkl")
df2=pd.read_pickle("tw_pos_score1.pkl")
df_pos=pd.concat([df1,df2],axis=0).reset_index(drop=True)

In [ ]:
df3=pd.read_pickle("tw_neg.pkl")
df4=pd.read_pickle("tw_neg_score1.pkl")
df_neg=pd.concat([df3,df4],axis=0).reset_index(drop=True)

In [ ]:
df_pos.to_csv("pos.csv",index=False)
df_neg.to_csv("neg.csv",index=False)

In [ ]:
df5=pd.read_csv("pos.csv")
df6=pd.read_csv("neg.csv")
df_final=pd.concat([df5,df6],axis=0)

In [ ]:
df_final["label"].value_counts()

In [ ]:
def quitar_acentos(texto):
    juntar=(" ").join(texto)
    juntar=juntar.replace("á","a")
    juntar=juntar.replace("é","e")
    juntar=juntar.replace("í","i")
    juntar=juntar.replace("ó","o")
    juntar=juntar.replace("ú","u")
    lista_final=juntar.split()
    return lista_final

In [ ]:
from nltk.corpus import stopwords
stop=stopwords.words('spanish')

In [ ]:
stop=quitar_acentos(stop)

In [ ]:
from sklearn.model_selection import train_test_split

tw_train, tw_test, label_train, label_test = train_test_split(df_final["tweet"], df_final['label'],
                                                              test_size=0.30,random_state=2020)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [ ]:
pipeline = Pipeline([
    ('bow', CountVectorizer(stop_words=stop)), 
    ('classifier', MultinomialNB()),  
])

In [ ]:
pipeline.fit(tw_train,label_train)

In [ ]:
predictions = pipeline.predict(tw_test)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
print(confusion_matrix(label_test,predictions))
print('\n')
print(classification_report(label_test,predictions))

In [ ]:
df=pd.read_pickle("final_df.pkl").reset_index(drop=True)

In [ ]:
predictions_final=pipeline.predict(df["tweet"])

In [ ]:
df["label"]=predictions_final

In [ ]:
df["label"].value_counts()

In [ ]:
df[df["label"]=="positivo"].sample(5)

In [ ]:
df[df["label"]=="negativo"].sample(5)

In [ ]:
df_sent=df.groupby(["mes","dia","label"]).count()["tweet"]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid",
        rc={
            "text.color": "black",
            "axes.labelcolor": "black",
            "axes.edgecolor": "black",
            "xtick.color": "black",
            "ytick.color": "black",
            "axes.facecolor": "#FFFFFF",
            "figure.facecolor": "#F5F8FA"}
        )

In [ ]:
df_sent.xs("positivo",level=2).plot(lw=2,ls="--",color="green",figsize=(20,10),label="Positivo")
df_sent.xs("negativo",level=2).plot(lw=2,ls="--",color="red",figsize=(20,10),label="Negativo")
plt.xticks(ticks=np.arange(0, df["fecha_str"].nunique()),labels=df["fecha_str"].unique(),size=20,rotation=90)
plt.yticks(np.arange(0, 1901 , 100),size=20)
plt.title("Sentimiento de los tweets",weight="bold",pad=15,size=22)
plt.xlabel("Fecha",weight="bold",labelpad=20,size=18)
plt.ylabel("Número de tweets",weight="bold",labelpad=20,size=18)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5),fontsize="x-large")
plt.tight_layout()